In [2]:
import sys
%load_ext autoreload
%autoreload 2
sys.path.append('..')
sys.path.append('../keras_bert/')

import os
import re
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
import torch
from lib import bert_model
from tqdm import tqdm_notebook as tqdm

Better speed can be achieved with apex installed from https://www.github.com/nvidia/apex.


In [3]:
%env CUDA_VISIBLE_DEVICES=1

NUM_LABELS = 374

params = {
    'num_labels': NUM_LABELS,
    'label_list': [str(label) for label in range(NUM_LABELS)],
    'output_dir': '../output',
    'cache_dir': '../model_cache',
    'bert_model': 'bert-base-multilingual-uncased',
    'max_seq_length': 128,
    'train_batch_size': 32,
    'eval_batch_size': 8,
    'learning_rate': 2e-5,
    'warmup_proportion': 0.1,
    'num_train_epochs': 2,
    'type': 'multilabel',
    'seed': 1331,
    'device': torch.device(
        'cuda' if torch.cuda.is_available()
        else 'cpu')
}

params['lower_case'] = (params['bert_model'][-7:] == 'uncased')

env: CUDA_VISIBLE_DEVICES=1


## Preparing data

In [ ]:
DATA_DIR = '../datasets'

df_train = pd.read_csv(
    os.path.join(DATA_DIR, 'data/train.csv'),
    encoding='utf-8',
)
df_valid = pd.read_csv(
    os.path.join(DATA_DIR, 'data/dev.csv'),
    encoding='utf-8',
)
df_test = pd.read_csv(
    os.path.join(DATA_DIR, 'data/test.csv'),
    encoding='utf-8',
)

In [6]:
def create_ohe_labels(labels_column):
    labels_str = re.sub('\s+', ' ', labels_column.strip('[] ')).split(' ')
    text_labels = np.array(labels_str, dtype=int)
    text_labels
    codes = np.zeros(params['num_labels'], dtype=int)
    codes[text_labels] = 1
    return codes.tolist()

def get_data_ohe_labels(df):
    targets = []
    for i, row in tqdm(df.iterrows(), total=len(df)):
        try:
            targets.append(create_ohe_labels(row['labels']))
        except:
            print(i)
            break
    return targets

In [7]:
y_train = get_data_ohe_labels(df_train)
y_valid = get_data_ohe_labels(df_valid)
y_test = get_data_ohe_labels(df_test)

In [8]:
X_train = df_train['text'].values
X_valid = df_valid['text'].values
X_test = df_test['text'].values

## Training BERT model

In [17]:
model = bert_model.BertTextClassificationModel(params)

Completed!


In [ ]:
result = model.fit(
    X_train,
    y_train,
    batch_size=32,
    n_epochs=50,
    validation_data=(X_valid, y_valid),
    best_model_output='model_saved.pth',
)
result


Epoch: 1


Iteration: 100%|██████████| 3273/3273 [26:31<00:00,  2.35it/s]


***** Running evaluation *****


Predicting: 100%|██████████| 3273/3273 [02:07<00:00, 26.15it/s]


{'train_log_loss': 0.048060563627119426, 'eval_accuracy': 0.0}

Epoch: 2


...

Predicting: 100%|██████████| 3273/3273 [02:06<00:00, 26.54it/s]


{'train_log_loss': 0.0015061203975090779, 'eval_accuracy': 0.5695897945153159}

Epoch: 30


Iteration: 100%|██████████| 3273/3273 [26:32<00:00,  2.16it/s]


***** Running evaluation *****


Predicting:  30%|███       | 996/3273 [00:38<01:27, 25.98it/s]

In [19]:
result, preds = model.evaluate(X_test, y_test, batch_size=8)

***** Running evaluation *****


Predicting: 100%|██████████| 4091/4091 [02:38<00:00, 25.79it/s]


In [20]:
result_val, preds_val = model.evaluate(X_valid, y_valid, batch_size=8)

***** Running evaluation *****


Predicting: 100%|██████████| 3273/3273 [02:07<00:00, 26.58it/s]


In [21]:
result, result_val

({'eval_accuracy': 0.5777187032114156}, {'eval_accuracy': 0.5781452906577038})

In [22]:
from sklearn import metrics
def print_metrics(y_true, preds):
    y_true = np.array(y_true)
    y_pred = np.array(preds >= 0.5, dtype=int)
    return {
        'micro_f1': metrics.f1_score(y_true, y_pred, average='micro'),
        'macro_f1': metrics.f1_score(y_true, y_pred, average='macro'),
        'accuracy': metrics.accuracy_score(y_true, y_pred),
        'hamming_accuracy': 1 - metrics.hamming_loss(y_true, y_pred)
    }

In [23]:
print_metrics(y_valid, preds_val)

/home/ramild/miniconda2/envs/py361/lib/python3.6/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/home/ramild/miniconda2/envs/py361/lib/python3.6/site-packages/sklearn/metrics/classification.py:1145: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


{'micro_f1': 0.6950016562994591,
 'macro_f1': 0.22566219109122998,
 'accuracy': 0.5781452906577038,
 'hamming_accuracy': 0.9966150153369033}

In [24]:
print_metrics(y_test, preds)

{'micro_f1': 0.6918780681383933,
 'macro_f1': 0.22607089294303334,
 'accuracy': 0.5777187032114156,
 'hamming_accuracy': 0.996574481257932}